# Use pre-trained VGG16 model

[example](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

In [1]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.utils.io_utils import HDF5Matrix
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [ ]:
data_path = '/data/metasearch-anatomical-brainmask-slices.h5'
start = 0
end = 100


X_train = HDF5Matrix(data_path, '/anatomical/axial', start=start, end=end)

In [ ]:
def process_slice(slice_):
    import scipy
    slice_ = scipy.misc.imresize(slice_, size=(299, 299))

In [ ]:
model_vgg16_conv = VGG16(include_top=False, weights='imagenet')
model_vgg16_conv.summary()